In [3]:
import rosbag
PATH="/home/xuhao/Dropbox/data/TRO2020-SwarmLocal/tro2020-bags/pre/"
PATHOUT="/home/xuhao/bags/swarm_local_2020_12/"
input_bag = PATH +"drone1-swarm_local_2020-12-13-21-31-51.bag"
output_bag = PATHOUT + "drone1-swarm_local_2020-12-13-21-31-51_processed.bag"
moveout_topics = {
    "/swarm_drones/node_detected",
    "/swarm_drones/swarm_drone_fused",
    "/swarm_drones/swarm_drone_fused_relative",
    "/swarm_drones/swarm_frame",
    "/swarm_drones/swarm_frame_predict"
}
#Moveout node_detected
with rosbag.Bag(output_bag, 'w') as outbag:
    for topic, msg, t in rosbag.Bag(input_bag).read_messages():
        if topic in moveout_topics:
            pass
        else:
            #outbag.write(topic, msg, msg.header.stamp if msg._has_header else t)
            outbag.write(topic, msg, t)            

In [17]:
import rosbag
import os
import rospy
#PATH="/home/xuhao/bags/swarm_local_2020_12/"
PATH="/media/xuhao/DataStorage/tro2020-data/"
PATHOUT="/home/xuhao/bags/swarm_local_2020_12/"
bags = ["drone1-swarm_local_raw_2020-12-22.bag",
                "drone2-swarm_local_raw_2020-12-22.bag"]
t_offset = 30
#t_offset = 90

print("Will use", t0, "as start")

t0 = 0
for bag in bags:
    for topic, msg, t in rosbag.Bag(PATH+bag).read_messages():
        print(f"Bag {bag} start at {t.to_sec()}")
        if t.to_sec() > t0:
            t0 = t.to_sec()
            break
        else:
            break

ros_t0 = None
lps_t1 = None
moveout_topics = {
    "/uwb_node/incoming_broadcast_data"
}

dt0 = 0
dt1 = 0
for topic, msg, t in rosbag.Bag(PATH+bags[0]).read_messages():
    if t.to_sec() > t0 + t_offset and topic == "/uwb_node/time_ref":
        _t0 = msg.header.stamp
        _t1 = msg.time_ref
        dt0 = _t0 - _t1
        break

for topic, msg, t in rosbag.Bag(PATH+bags[1]).read_messages():
    if  t.to_sec() > t0 + t_offset and topic == "/uwb_node/time_ref":
        _t0 = msg.header.stamp
        _t1 = msg.time_ref
        dt1 = _t0 - _t1
        break

            
dt = (dt1 - dt0)
print(f"Dt between drones {dt.to_sec()}", dt1, dt0)

for bag in bags:
    print(f"Converting {bag}")
    #cmd = f"/opt/ros/melodic/bin/rosbag filter {PATH}{bag} {PATHOUT}filtered-{bag} 't.to_sec() >= {t}'"
    #print(os.popen(cmd).read())
    _dt = rospy.Time(0)
    if bag != bags[0]:
        _dt = dt
    output_bag = PATHOUT+"filtered-"+bag;
    with rosbag.Bag(output_bag, 'w') as outbag:
        for topic, msg, t in rosbag.Bag(PATH+bag).read_messages():
            if topic in moveout_topics:
                pass
            else:
                t = msg.header.stamp - _dt if msg._has_header else t - _dt
                if msg._has_header:
                    msg.header.stamp = msg.header.stamp - _dt
                if t.to_sec() > t0 + t_offset:
                    outbag.write(topic, msg, t)

Will use 1608626220.2636192 as start
Bag drone1-swarm_local_raw_2020-12-22.bag start at 1608626218.627621
Bag drone2-swarm_local_raw_2020-12-22.bag start at 1608626220.2636192
Dt between drones -0.247542944 1608625031914662272 1608625032162205216
Converting drone1-swarm_local_raw_2020-12-22.bag
Converting drone2-swarm_local_raw_2020-12-22.bag


In [ ]:
ros_t0 = None
lps_t1 = None
for bag in bags:
    output_bag = PATHOUT+"filtered-"+bag;
    with rosbag.Bag(output_bag, 'w') as outbag:
        for topic, msg, t in rosbag.Bag(PATH+bag).read_messages():
            if t < t0:
                continue
            if bag ==  "drone2-swarm_local_raw_2020-12-12.bag":
                if topic == "/uwb_node/remote_nodes":
                    if ros_t0 is None:
                        ros_t0 = msg.header.stamp.to_sec()
                    if lps_t1 is None:
                        lps_t1 = msg.sys_time
                    lps_time = (msg.header.stamp.to_sec() - ros_t0)*1000 + lps_t1
                    #print(msg.sys_time, lps_time)
                    msg.sys_time = int(lps_time)
            outbag.write(topic, msg, msg.header.stamp if msg._has_header else t)